In [1]:
# ============================================================
# Vietnamese Speaker Profiling - Kaggle Training
# ============================================================
!git clone https://github.com/VuThanhLam124/Profiling_gender_dialect.git

Cloning into 'Profiling_gender_dialect'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (58/58), done.
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 88 (delta 43), reused 74 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (88/88), 88.91 KiB | 1.27 MiB/s, done.
Resolving deltas: 100% (43/43), done.
remote: Total 88 (delta 43), reused 74 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (88/88), 88.91 KiB | 1.27 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
cd Profiling_gender_dialect

/kaggle/working/Profiling_gender_dialect


In [3]:
ls

app.py               eval.py      notebooks/       requirements.txt
compare_encoders.py  finetune.py  prepare_data.py  src/
configs/             infer.py     README.md


In [4]:
# Install dependencies
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/68.6 kB ? eta -:--:--Collecting pydantic<2.12,>=2.0 (from gradio>=4.0.0->-r requirements.txt (line 13))
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/363.4 MB 37.6 MB/s eta 0:00:10Downloading nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━╺━━━━━

In [ ]:
# Check dataset structure and metadata columns
import os
import pandas as pd

VISPEECH_ROOT = "/kaggle/input/vispeech"

print("Dataset structure:")
for item in os.listdir(VISPEECH_ROOT):
    item_path = os.path.join(VISPEECH_ROOT, item)
    if os.path.isdir(item_path):
        print(f"  {item}/")
        for subitem in os.listdir(item_path)[:3]:
            print(f"    - {subitem}")
    else:
        print(f"  {item}")

# Check metadata format
print("\n" + "=" * 50)
print("METADATA INFO:")
print("=" * 50)
meta_path = os.path.join(VISPEECH_ROOT, "metadata/trainset.csv")
df = pd.read_csv(meta_path)
print(f"Columns: {list(df.columns)}")
print(f"Total samples: {len(df)}")
print(f"\nFirst 3 rows:")
print(df.head(3).to_string())

# Check column names for finetune.py compatibility
print("\n" + "=" * 50)
print("COLUMN MAPPING CHECK:")
print("=" * 50)
required_cols = {
    'audio_name': ['audio_name', 'filename', 'file', 'path'],
    'gender': ['gender', 'sex'],
    'dialect': ['dialect', 'accent', 'region'],
    'speaker': ['speaker', 'speaker_id', 'spk_id']
}

for target, candidates in required_cols.items():
    found = None
    for col in candidates:
        if col in df.columns:
            found = col
            break
    if found:
        print(f"[OK] {target}: found as '{found}'")
    else:
        print(f"[MISSING] {target}: NOT FOUND (need one of {candidates})")

In [ ]:
# ============================================================
# TEST 3 ENCODER TYPES (5 epochs each)
# ============================================================
import yaml
import os

ENCODERS_TO_TEST = [
    "microsoft/wavlm-base-plus",
    "facebook/hubert-base-ls960",
    "facebook/wav2vec2-base",
]

base_config = """
# Test Configuration - Full Model Finetuning
model:
  name: "{encoder_name}"
  num_genders: 2
  num_dialects: 3
  dropout: 0.15
  head_hidden_dim: 256
  freeze_encoder: false  # MUST BE FALSE for full finetuning

training:
  batch_size: 8
  gradient_accumulation_steps: 4
  learning_rate: 5e-5
  num_epochs: 5  # Quick test
  warmup_ratio: 0.125
  weight_decay: 0.0125
  gradient_clip: 1.0
  lr_scheduler: "linear"
  fp16: true
  dataloader_num_workers: 2

loss:
  dialect_weight: 2.5

mlflow:
  enabled: false

data:
  train_meta: "/kaggle/input/vispeech/metadata/trainset.csv"
  train_audio: "/kaggle/input/vispeech/trainset"
  val_split: 0.15

audio:
  sampling_rate: 16000
  max_duration: 5

augmentation:
  enabled: true
  prob: 0.8

output:
  dir: "/kaggle/working/output_{encoder_short}"
  save_total_limit: 1
  metric_for_best_model: "dialect_acc"

early_stopping:
  patience: 3
  threshold: 0.001

labels:
  gender:
    Male: 0
    Female: 1
  dialect:
    North: 0
    Central: 1
    South: 2

seed: 42
"""

results = {}

for encoder in ENCODERS_TO_TEST:
    encoder_short = encoder.split("/")[-1]
    print("\n" + "=" * 70)
    print(f"TESTING: {encoder}")
    print("=" * 70)
    
    # Create config for this encoder
    config_content = base_config.format(
        encoder_name=encoder,
        encoder_short=encoder_short
    )
    
    config_path = f"configs/test_{encoder_short}.yaml"
    with open(config_path, "w") as f:
        f.write(config_content)
    
    print(f"Config saved: {config_path}")
    print(f"freeze_encoder: false (full finetuning)")
    print(f"Training 5 epochs...")
    
    # Run training
    exit_code = os.system(f"python finetune.py --config {config_path}")
    
    if exit_code == 0:
        results[encoder] = "SUCCESS"
        print(f"\n{encoder_short}: Training completed!")
    else:
        results[encoder] = f"FAILED (exit code: {exit_code})"
        print(f"\n{encoder_short}: Training failed!")

# Summary
print("\n" + "=" * 70)
print("TEST SUMMARY")
print("=" * 70)
for encoder, status in results.items():
    print(f"  {encoder}: {status}")

2025-11-28 02:07:20.224680: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764295640.246499     223 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764295640.252910     223 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
E0000 00:00:1764295640.246499     223 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764295640.252910     223 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeErr

In [43]:
# Check saved model
import os

output_model_dir = "/kaggle/working/output/best_model"
if os.path.exists(output_model_dir):
    print(f"Model saved at: {output_model_dir}")
    print("Files:")
    for f in os.listdir(output_model_dir):
        size = os.path.getsize(os.path.join(output_model_dir, f)) / 1024 / 1024
        print(f"  - {f} ({size:.2f} MB)")
else:
    print("Model not found. Check training logs.")

Model saved at: /kaggle/working/output/best_model
Files:
  - training_args.bin (0.01 MB)
  - model.safetensors (3.89 MB)


In [ ]:
# ============================================================
# EVALUATION - Evaluate best model on test sets  
# ============================================================
# Sử dụng raw audio mode (không cần extract features)
!python eval.py \
    --checkpoint /kaggle/working/output/best_model \
    --config configs/finetune.yaml \
    --test_name clean_test \
    --test_name2 noisy_test \
    --output_dir /kaggle/working/output/evaluation